In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2024-us-presidential-elections-twitter-data/Republican_raw.csv
/kaggle/input/2024-us-presidential-elections-twitter-data/Democrat_raw.csv
/kaggle/input/2024-us-presidential-elections-twitter-data/final_raw.csv
/kaggle/input/2024-us-presidential-elections-twitter-data/preprocessed_tweets_us24.csv
/kaggle/input/2024-us-presidential-elections-twitter-data/preprocessedtranslated_tweets_us24.csv


In [2]:
import pandas as pd

# Load the CSV file into a DataFrame
df_republican = pd.read_csv('/kaggle/input/2024-us-presidential-elections-twitter-data/Republican_raw.csv')

# Display the first few rows
df_republican.head()


,Tweet ID,Username,Text,Date,Retweets,Likes,Weight
0,1798142683145543755,Geraldine Speer,Republican lawmakers don’t know the law 😶,Tue Jun 04 23:59:58 +0000 2024,0,0,0.970688
1,1798142665579753806,Predato- Trump,@RepMTG You are the do nothing republican. You...,Tue Jun 04 23:59:54 +0000 2024,0,0,0.970688
2,1798142642750115977,coolguy,@Root25257968 @sanidhyakny @d0nut2x When did i...,Tue Jun 04 23:59:48 +0000 2024,0,4,0.993307
3,1798142637175718336,Ron cohen,Fuming Marjorie Taylor Greene Agrees the Repub...,Tue Jun 04 23:59:47 +0000 2024,0,1,0.500000
4,1798142631501054172,chris w,@GenoVeno73 @GOP @HouseGOP @SenateGOP @Jim_Jor...,Tue Jun 04 23:59:45 +0000 2024,0,0,0.880797


In [3]:
import re

def preprocess_tweet(text):
    text = str(text)
    text = text.lower()  # lowercase
    text = re.sub(r'http\S+|www\S+', '', text)  # remove URLs
    text = re.sub(r'@\w+', '', text)  # remove mentions
    text = re.sub(r'#\w+', '', text)  # remove hashtags
    text = re.sub(r'\n', ' ', text)  # remove newlines
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    text = re.sub(r'\d+', '', text)  # remove numbers
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra whitespace
    return text


In [6]:
!pip install -q transformers

from transformers import pipeline

# Apply preprocessing to text column
df_republican['clean_text'] = df_republican['Text'].apply(preprocess_tweet)

sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

# Then run sentiment analysis on the cleaned text
df_republican['sentiment'] = df_republican['clean_text'].apply(lambda x: sentiment_pipeline(x)[0]['label'])

# Optional: map sentiment labels
label_map = {
    'LABEL_0': 'negative',
    'LABEL_1': 'neutral',
    'LABEL_2': 'positive'
}
df_republican['sentiment'] = df_republican['sentiment'].map(label_map)


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

Device set to use cpu
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [7]:
# Save the DataFrame to a new CSV file
df_republican.to_csv('Republican_with_sentiment.csv', index=False)


In [ ]:
df_democrat = pd.read_csv('/kaggle/input/2024-us-presidential-elections-twitter-data/Democrat_raw.csv')
df_democrat['clean_text'] = df_democrat['Text'].apply(preprocess_tweet)

sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

# Then run sentiment analysis on the cleaned text
df_democrat['sentiment'] = df_democrat['clean_text'].apply(lambda x: sentiment_pipeline(x)[0]['label'])

# Optional: map sentiment labels
label_map = {
    'LABEL_0': 'negative',
    'LABEL_1': 'neutral',
    'LABEL_2': 'positive'
}
df_democrat['sentiment'] = df_democrat['sentiment'].map(label_map)
df_democrat.to_csv('Democrat_with_sentiment.csv', index=False)


Device set to use cpu
